## 311 Unified Data Analysis
---
### Data Source and API

- AODP Dataset Access: https://data.austintexas.gov/City-Government/311-Unified-Data-Test-Site-2019/i26j-ai4z
- API Endpoint: https://data.austintexas.gov/resource/i26j-ai4z.json
- API Documentation: https://dev.socrata.com/foundry/data.austintexas.gov/i26j-ai4z

Column Name meaning -
Service Request (SR) Number: The service request tracking number.


### Data Extraction and Munging

In [2]:
# Dependencies and packages
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math as math
import datetime as dt
#import seaborn as sns
import pandas as pd
from sodapy import Socrata

In [8]:
client = Socrata("data.austintexas.gov", None)

results = client.get("i26j-ai4z", limit=8500000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df

,sr_number,sr_type_code,sr_type_desc,sr_department_desc,sr_method_received_desc,sr_status_desc,sr_status_date,sr_created_date,sr_updated_date,sr_location,...,sr_location_map_tile,:@computed_region_jcrc_4uuy,:@computed_region_m2th_e4b7,:@computed_region_rxpj_nzrk,:@computed_region_a3it_2a2z,sr_location_council_district,:@computed_region_8spj_utxs,:@computed_region_q9nd_rr82,:@computed_region_e9j2_6w3z,sr_closed_date
0,20-00019015,CODECOMP,Austin Code - Request Code Officer,Austin Code Department,Spot311 Interface,Open,2020-01-15T19:28:06.000,2020-01-15T19:28:06.000,2020-01-15T19:28:06.000,"15806 JACOBSON RD, AUSTIN, TX 78617",...,MR10,45,484,45,8002,NaN,NaN,NaN,NaN,NaN
1,20-00019014,CODECOMP,Austin Code - Request Code Officer,Austin Code Department,Spot311 Interface,Open,2020-01-15T19:28:02.000,2020-01-15T19:28:02.000,2020-01-15T19:28:02.000,"4650 E SH 71 EB, AUSTIN, TX 78617",...,MR15,45,484,45,8002,NaN,NaN,NaN,NaN,NaN
2,20-00019013,STREETL2,Street Light Issue- Address,Austin Energy Department,Phone,New,2020-01-15T19:27:21.000,2020-01-15T19:27:21.000,2020-01-15T19:27:21.000,"6001 GUADALUPE ST, AUSTIN, TX 78752",...,MK27,40,217,3,3641,4,4,9,77,NaN
3,20-00019010,PWSIDEWL,Sidewalk/Curb Ramp/Route - NEW,Public Works,Phone,New,2020-01-15T19:23:48.000,2020-01-15T19:23:48.000,2020-01-15T19:23:48.000,"6904 TWILIGHT MESA DR, AUSTIN, TX 78749",...,MB18,38,531,53,3638,8,8,2,73,NaN
4,20-00019009,CODECOMP,Austin Code - Request Code Officer,Austin Code Department,Spot311 Interface,Open,2020-01-15T19:22:09.000,2020-01-15T19:22:09.000,2020-01-15T19:22:09.000,"2266 W BEN WHITE BLVD WB, AUSTIN, TX 78704",...,MF19,9,179,75,2859,5,5,1,22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862534,14-00000294,TRASIGMA,Traffic Signal - Maintenance,Transportation,Phone,Closed,2014-07-23T13:06:48.000,2014-01-01T07:02:20.000,2014-07-23T13:06:49.000,"MANOR RD & SPRINGDALE RD, AUSTIN, TX",...,MM26,19,309,33,3239,NaN,1,8,41,2014-07-23T13:06:48.000
862535,14-00000268,ACCOYTE,Coyote Complaints,Animal Services Office,Phone,Closed,2014-01-05T07:43:52.000,2014-01-01T04:54:56.000,2014-01-05T07:43:52.000,"8600 W SH 71, AUSTIN, TX",...,MB20,32,295,49,3251,NaN,8,2,73,2014-01-05T07:43:52.000
862536,14-00000261,SIGNTRA1,Sign - Traffic Sign Maintenance,Transportation,Phone,Closed,2014-01-03T06:57:22.000,2014-01-01T04:25:34.000,2014-01-03T06:57:22.000,"1411 MEARNS MEADOW BLVD, AUSTIN, TX",...,ML31,22,217,18,3646,NaN,4,9,16,2014-01-03T06:57:22.000
862537,14-00000230,ZZANIMAL,Lost Item in Storm Drainage System,Watershed Protection,Phone,Closed,2014-01-03T07:40:49.000,2014-01-01T03:02:45.000,2014-01-03T07:40:50.000,"W 6TH ST & LAVACA ST, AUSTIN, TX",...,MJ22,52,217,24,2856,NaN,9,10,60,2014-01-03T07:40:49.000


In [10]:
#df = pd.read_csv('Data_Files/311_Unified_Data_Test_Site_2019.csv', low_memory=False)
#df_backup = df

In [11]:
# Data Filtering to only store complaints about Service request type 'DOCKMOBI' which 
# includes SR Descriptions 'Shared Micromobility' and 'Dockless Mobility'.

results_df = results_df[results_df["sr_type_code"]== "DOCKMOBI"]
results_df.head()

,sr_number,sr_type_code,sr_type_desc,sr_department_desc,sr_method_received_desc,sr_status_desc,sr_status_date,sr_created_date,sr_updated_date,sr_location,...,sr_location_map_tile,:@computed_region_jcrc_4uuy,:@computed_region_m2th_e4b7,:@computed_region_rxpj_nzrk,:@computed_region_a3it_2a2z,sr_location_council_district,:@computed_region_8spj_utxs,:@computed_region_q9nd_rr82,:@computed_region_e9j2_6w3z,sr_closed_date
9,20-00018990,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Open,2020-01-15T19:01:04.000,2020-01-15T19:01:04.000,2020-01-15T19:01:04.000,"1711 RUTLAND DR, AUSTIN, TX 78758",...,MK31,22,217,18,3646,4,4,9,16,NaN
13,20-00018982,DOCKMOBI,Shared Micromobility,Transportation,Phone,Open,2020-01-15T18:52:56.000,2020-01-15T18:52:56.000,2020-01-15T18:52:56.000,"1810 BARTON SPRINGS RD, AUSTIN, TX 78704",...,MH22,9,217,75,2859,5,5,1,2,NaN
15,20-00018974,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Open,2020-01-15T18:43:03.000,2020-01-15T18:43:03.000,2020-01-15T18:43:03.000,"1711 RUTLAND DR, AUSTIN, TX 78758",...,MK31,22,217,18,3646,4,4,9,16,NaN
31,20-00018925,DOCKMOBI,Shared Micromobility,Transportation,Phone,Open,2020-01-15T17:58:35.000,2020-01-15T17:58:35.000,2020-01-15T17:58:35.000,"1030 BARTON HILLS DR, AUSTIN, TX 78704",...,MG21,9,217,75,2859,5,5,1,93,NaN
32,20-00018922,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Open,2020-01-15T17:58:00.000,2020-01-15T17:58:00.000,2020-01-15T17:58:00.000,"1711 RUTLAND DR, AUSTIN, TX 78758",...,MK31,22,217,18,3646,4,4,9,16,NaN


In [12]:
results_df.count()

sr_number                       10147
sr_type_code                    10147
sr_type_desc                    10147
sr_department_desc              10147
sr_method_received_desc         10147
sr_status_desc                  10147
sr_status_date                  10147
sr_created_date                 10147
sr_updated_date                 10147
sr_location                     10147
sr_location_street_number        9001
sr_location_street_name         10147
sr_location_city                10144
sr_location_zip_code            10069
sr_location_county              10136
sr_location_x                   10147
sr_location_y                   10147
sr_location_lat                 10147
sr_location_long                10147
sr_location_lat_long            10147
sr_location_map_page            10136
sr_location_map_tile            10135
:@computed_region_jcrc_4uuy     10135
:@computed_region_m2th_e4b7     10087
:@computed_region_rxpj_nzrk     10140
:@computed_region_a3it_2a2z     10140
sr_location_

In [13]:
# Check how many sub types (SR Type Desc) inclue SR Type Code "DOCKMOBI"
results_df["sr_type_desc"].value_counts()

Dockless Mobility       6454
Shared Micromobility    3693
Name: sr_type_desc, dtype: int64

In [14]:
results_df.count()

sr_number                       10147
sr_type_code                    10147
sr_type_desc                    10147
sr_department_desc              10147
sr_method_received_desc         10147
sr_status_desc                  10147
sr_status_date                  10147
sr_created_date                 10147
sr_updated_date                 10147
sr_location                     10147
sr_location_street_number        9001
sr_location_street_name         10147
sr_location_city                10144
sr_location_zip_code            10069
sr_location_county              10136
sr_location_x                   10147
sr_location_y                   10147
sr_location_lat                 10147
sr_location_long                10147
sr_location_lat_long            10147
sr_location_map_page            10136
sr_location_map_tile            10135
:@computed_region_jcrc_4uuy     10135
:@computed_region_m2th_e4b7     10087
:@computed_region_rxpj_nzrk     10140
:@computed_region_a3it_2a2z     10140
sr_location_

### Data Cleaning

In [15]:
clean_df = results_df.copy()

In [16]:
# Renaming Column Names
clean_df.columns = clean_df.columns.str.replace('_',' ').str.title().str.replace('Sr','SR')
clean_df

,SR Number,SR Type Code,SR Type Desc,SR Department Desc,SR Method Received Desc,SR Status Desc,SR Status Date,SR Created Date,SR Updated Date,SR Location,...,SR Location Map Tile,:@Computed Region Jcrc 4Uuy,:@Computed Region M2Th E4B7,:@Computed Region Rxpj Nzrk,:@Computed Region A3It 2A2Z,SR Location Council District,:@Computed Region 8Spj Utxs,:@Computed Region Q9Nd Rr82,:@Computed Region E9J2 6W3Z,SR Closed Date
9,20-00018990,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Open,2020-01-15T19:01:04.000,2020-01-15T19:01:04.000,2020-01-15T19:01:04.000,"1711 RUTLAND DR, AUSTIN, TX 78758",...,MK31,22,217,18,3646,4,4,9,16,NaN
13,20-00018982,DOCKMOBI,Shared Micromobility,Transportation,Phone,Open,2020-01-15T18:52:56.000,2020-01-15T18:52:56.000,2020-01-15T18:52:56.000,"1810 BARTON SPRINGS RD, AUSTIN, TX 78704",...,MH22,9,217,75,2859,5,5,1,2,NaN
15,20-00018974,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Open,2020-01-15T18:43:03.000,2020-01-15T18:43:03.000,2020-01-15T18:43:03.000,"1711 RUTLAND DR, AUSTIN, TX 78758",...,MK31,22,217,18,3646,4,4,9,16,NaN
31,20-00018925,DOCKMOBI,Shared Micromobility,Transportation,Phone,Open,2020-01-15T17:58:35.000,2020-01-15T17:58:35.000,2020-01-15T17:58:35.000,"1030 BARTON HILLS DR, AUSTIN, TX 78704",...,MG21,9,217,75,2859,5,5,1,93,NaN
32,20-00018922,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Open,2020-01-15T17:58:00.000,2020-01-15T17:58:00.000,2020-01-15T17:58:00.000,"1711 RUTLAND DR, AUSTIN, TX 78758",...,MK31,22,217,18,3646,4,4,9,16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260696,18-00220040,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2018-09-26T14:42:44.000,2018-07-10T09:05:35.000,2018-09-26T14:42:44.000,"122 FREDERICK ST, AUSTIN, TX 78704",...,MH19,9,217,75,2859,3,3,3,47,2018-09-26T14:42:44.000
262317,18-00215309,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2018-09-26T14:46:37.000,2018-07-06T11:42:03.000,2018-09-26T14:46:37.000,"901 CHICON ST, AUSTIN, TX 78702",...,MK22,7,309,1,2857,1,1,8,59,2018-09-26T14:46:37.000
262603,18-00214399,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2018-09-26T14:50:43.000,2018-07-05T18:11:22.000,2018-09-26T14:50:43.000,"1609 ENFIELD RD, AUSTIN, TX 78703",...,MH23,26,126,69,2858,9,9,10,72,2018-09-26T14:50:43.000
263303,18-00211272,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2019-01-09T10:09:41.000,2018-07-04T11:21:35.000,2019-01-09T10:09:41.000,"1172 WALLER ST, AUSTIN, TX 78702",...,MK22,7,309,1,2857,1,1,8,56,2019-01-09T10:09:41.000


In [17]:
clean_df.count()

SR Number                       10147
SR Type Code                    10147
SR Type Desc                    10147
SR Department Desc              10147
SR Method Received Desc         10147
SR Status Desc                  10147
SR Status Date                  10147
SR Created Date                 10147
SR Updated Date                 10147
SR Location                     10147
SR Location Street Number        9001
SR Location Street Name         10147
SR Location City                10144
SR Location Zip Code            10069
SR Location County              10136
SR Location X                   10147
SR Location Y                   10147
SR Location Lat                 10147
SR Location Long                10147
SR Location Lat Long            10147
SR Location Map Page            10136
SR Location Map Tile            10135
:@Computed Region Jcrc 4Uuy     10135
:@Computed Region M2Th E4B7     10087
:@Computed Region Rxpj Nzrk     10140
:@Computed Region A3It 2A2Z     10140
SR Location 

In [18]:
# Delete unwanted columns:
clean_df = clean_df[clean_df.columns[~clean_df.columns.str.contains(':@Computed Region')]]
list(clean_df.columns)

['SR Number',
 'SR Type Code',
 'SR Type Desc',
 'SR Department Desc',
 'SR Method Received Desc',
 'SR Status Desc',
 'SR Status Date',
 'SR Created Date',
 'SR Updated Date',
 'SR Location',
 'SR Location Street Number',
 'SR Location Street Name',
 'SR Location City',
 'SR Location Zip Code',
 'SR Location County',
 'SR Location X',
 'SR Location Y',
 'SR Location Lat',
 'SR Location Long',
 'SR Location Lat Long',
 'SR Location Map Page',
 'SR Location Map Tile',
 'SR Location Council District',
 'SR Closed Date']

In [19]:
# Dropping all the null value rows:
clean_df = clean_df.dropna(how='any')
clean_df.head()

,SR Number,SR Type Code,SR Type Desc,SR Department Desc,SR Method Received Desc,SR Status Desc,SR Status Date,SR Created Date,SR Updated Date,SR Location,...,SR Location County,SR Location X,SR Location Y,SR Location Lat,SR Location Long,SR Location Lat Long,SR Location Map Page,SR Location Map Tile,SR Location Council District,SR Closed Date
361,20-00017969,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2020-01-15T11:28:09.000,2020-01-15T09:54:54.000,2020-01-15T11:28:09.000,"2506 LEON ST, AUSTIN, TX 78705",...,TRAVIS,3112102.43850464,10078748.74169090,30.29018509,-97.74921878,"{'latitude': '30.29018509', 'longitude': '-97....",585E,MJ24,9,2020-01-15T11:28:09.000
365,20-00017960,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2020-01-15T10:38:36.000,2020-01-15T09:51:45.000,2020-01-15T10:38:36.000,"520 W 6TH ST, AUSTIN, TX 78701",...,TRAVIS,3112623.74878748,10071339.00205350,30.26978352,-97.74811306,"{'latitude': '30.26978352', 'longitude': '-97....",585S,MJ22,9,2020-01-15T10:38:36.000
470,20-00017679,DOCKMOBI,Shared Micromobility,Transportation,Phone,Closed,2020-01-15T09:43:58.000,2020-01-15T07:46:43.000,2020-01-15T09:43:58.000,"3805 GUADALUPE ST, AUSTIN, TX 78751",...,TRAVIS,3115570.52769994,10083453.93320070,30.30289697,-97.73788572,"{'latitude': '30.30289697', 'longitude': '-97....",555X,MJ25,9,2020-01-15T09:43:58.000
543,20-00017495,DOCKMOBI,Shared Micromobility,Transportation,Phone,Closed,2020-01-15T09:47:07.000,2020-01-14T21:57:15.000,2020-01-15T09:47:07.000,"1124 W RIVERSIDE DR, AUSTIN, TX 78704",...,TRAVIS,3110217.95695843,10069284.15064750,30.26428853,-97.75588269,"{'latitude': '30.26428853', 'longitude': '-97....",584Z,MH22,5,2020-01-15T09:47:07.000
547,20-00017490,DOCKMOBI,Shared Micromobility,Transportation,Phone,Closed,2020-01-15T09:48:24.000,2020-01-14T21:45:04.000,2020-01-15T09:48:24.000,"214 WEST AVE, AUSTIN, TX 78701",...,TRAVIS,3111465.93843056,10070330.98505740,30.26708656,-97.75185379,"{'latitude': '30.26708656', 'longitude': '-97....",584Z,MH22,9,2020-01-15T09:48:24.000


In [20]:
# Check how many null value got deleted
clean_df.count()

SR Number                       8799
SR Type Code                    8799
SR Type Desc                    8799
SR Department Desc              8799
SR Method Received Desc         8799
SR Status Desc                  8799
SR Status Date                  8799
SR Created Date                 8799
SR Updated Date                 8799
SR Location                     8799
SR Location Street Number       8799
SR Location Street Name         8799
SR Location City                8799
SR Location Zip Code            8799
SR Location County              8799
SR Location X                   8799
SR Location Y                   8799
SR Location Lat                 8799
SR Location Long                8799
SR Location Lat Long            8799
SR Location Map Page            8799
SR Location Map Tile            8799
SR Location Council District    8799
SR Closed Date                  8799
dtype: int64

Count after clean-up:

Looks like there were roughly ~130 rows with some NaN values for most columns deleted. 

In [21]:
# df for only the Shared micromobility Type Desc Complaints
shared_mobility_calls = clean_df[clean_df["SR Type Desc"] == "Shared Micromobility"]
shared_mobility_calls

,SR Number,SR Type Code,SR Type Desc,SR Department Desc,SR Method Received Desc,SR Status Desc,SR Status Date,SR Created Date,SR Updated Date,SR Location,...,SR Location County,SR Location X,SR Location Y,SR Location Lat,SR Location Long,SR Location Lat Long,SR Location Map Page,SR Location Map Tile,SR Location Council District,SR Closed Date
361,20-00017969,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2020-01-15T11:28:09.000,2020-01-15T09:54:54.000,2020-01-15T11:28:09.000,"2506 LEON ST, AUSTIN, TX 78705",...,TRAVIS,3112102.43850464,10078748.74169090,30.29018509,-97.74921878,"{'latitude': '30.29018509', 'longitude': '-97....",585E,MJ24,9,2020-01-15T11:28:09.000
365,20-00017960,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2020-01-15T10:38:36.000,2020-01-15T09:51:45.000,2020-01-15T10:38:36.000,"520 W 6TH ST, AUSTIN, TX 78701",...,TRAVIS,3112623.74878748,10071339.00205350,30.26978352,-97.74811306,"{'latitude': '30.26978352', 'longitude': '-97....",585S,MJ22,9,2020-01-15T10:38:36.000
470,20-00017679,DOCKMOBI,Shared Micromobility,Transportation,Phone,Closed,2020-01-15T09:43:58.000,2020-01-15T07:46:43.000,2020-01-15T09:43:58.000,"3805 GUADALUPE ST, AUSTIN, TX 78751",...,TRAVIS,3115570.52769994,10083453.93320070,30.30289697,-97.73788572,"{'latitude': '30.30289697', 'longitude': '-97....",555X,MJ25,9,2020-01-15T09:43:58.000
543,20-00017495,DOCKMOBI,Shared Micromobility,Transportation,Phone,Closed,2020-01-15T09:47:07.000,2020-01-14T21:57:15.000,2020-01-15T09:47:07.000,"1124 W RIVERSIDE DR, AUSTIN, TX 78704",...,TRAVIS,3110217.95695843,10069284.15064750,30.26428853,-97.75588269,"{'latitude': '30.26428853', 'longitude': '-97....",584Z,MH22,5,2020-01-15T09:47:07.000
547,20-00017490,DOCKMOBI,Shared Micromobility,Transportation,Phone,Closed,2020-01-15T09:48:24.000,2020-01-14T21:45:04.000,2020-01-15T09:48:24.000,"214 WEST AVE, AUSTIN, TX 78701",...,TRAVIS,3111465.93843056,10070330.98505740,30.26708656,-97.75185379,"{'latitude': '30.26708656', 'longitude': '-97....",584Z,MH22,9,2020-01-15T09:48:24.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82444,19-00281614,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2019-08-02T13:09:49.000,2019-07-24T20:23:08.000,2019-08-02T13:09:49.000,"115 SANDRA MURAIDA WAY, AUSTIN, TX 78703",...,TRAVIS,3110484.93366245,10070478.857265,30.26755559,-97.75494956,"{'latitude': '30.26755559', 'longitude': '-97....",584Z,MH22,9,2019-08-02T13:09:49.000
82495,19-00281506,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2019-07-31T09:45:28.000,2019-07-24T18:55:41.000,2019-07-31T09:45:28.000,"82 RAINEY ST, AUSTIN, TX 78701",...,TRAVIS,3115731.7473688,10067722.1322451,30.25964236,-97.7385377,"{'latitude': '30.25964236', 'longitude': '-97....",615B,MJ21,9,2019-07-31T09:45:28.000
82816,19-00280629,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2019-07-31T09:08:40.000,2019-07-24T10:18:53.000,2019-07-31T09:08:40.000,"404 W 26TH ST, AUSTIN, TX 78705",...,TRAVIS,3114522.92588404,10078930.902609,30.290531,-97.74153837,"{'latitude': '30.290531', 'longitude': '-97.74...",585E,MJ24,9,2019-07-31T09:08:40.000
82855,19-00280569,DOCKMOBI,Shared Micromobility,Transportation,Spot311 Interface,Closed,2019-07-31T09:10:05.000,2019-07-24T09:57:56.000,2019-07-31T09:10:05.000,"514 W 24TH ST, AUSTIN, TX 78705",...,TRAVIS,3114075.47189998,10077983.913823,30.28795653,-97.74302556,"{'latitude': '30.28795653', 'longitude': '-97....",585E,MJ24,9,2019-07-31T09:10:05.000


In [22]:
# df for only the Dockless mobility Type Desc Complaints
dockless_mobility_calls = clean_df[clean_df["SR Type Desc"] == "Dockless Mobility"]
dockless_mobility_calls

,SR Number,SR Type Code,SR Type Desc,SR Department Desc,SR Method Received Desc,SR Status Desc,SR Status Date,SR Created Date,SR Updated Date,SR Location,...,SR Location County,SR Location X,SR Location Y,SR Location Lat,SR Location Long,SR Location Lat Long,SR Location Map Page,SR Location Map Tile,SR Location Council District,SR Closed Date
79626,19-00288899,DOCKMOBI,Dockless Mobility,Transportation,Spot311 Interface,Closed,2019-07-30T14:03:48.000,2019-07-30T12:58:38.000,2019-07-30T14:03:48.000,"100 E 11TH ST, AUSTIN, TX 78701",...,TRAVIS,3114817.14974599,10072486.282712,30.27279688,-97.74108219,"{'latitude': '30.27279688', 'longitude': '-97....",585S,MJ22,1,2019-07-30T14:03:48.000
79644,19-00288862,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2019-07-30T14:02:26.000,2019-07-30T12:30:41.000,2019-07-30T14:02:26.000,"210 E 32ND ST, AUSTIN, TX 78705",...,TRAVIS,3116571.49903776,10080928.0004494,30.29588933,-97.73490165,"{'latitude': '30.29588933', 'longitude': '-97....",585B,MJ24,9,2019-07-30T14:02:26.000
79697,19-00288750,DOCKMOBI,Dockless Mobility,Transportation,Spot311 Interface,Closed,2019-07-30T12:12:55.000,2019-07-30T11:22:32.000,2019-07-30T12:12:55.000,"2002 S CONGRESS AVE, AUSTIN, TX 78704",...,TRAVIS,3111696.16883165,10061994.2417146,30.24415543,-97.75173749,"{'latitude': '30.24415543', 'longitude': '-97....",614R,MH20,9,2019-07-30T12:12:55.000
79803,19-00288519,DOCKMOBI,Dockless Mobility,Transportation,Spot311 Interface,Closed,2019-07-30T11:08:30.000,2019-07-30T09:46:21.000,2019-07-30T11:08:30.000,"7005 SHOAL CREEK BLVD, AUSTIN, TX 78757",...,TRAVIS,3113573.99770608,10100448.9982312,30.34974186,-97.74295668,"{'latitude': '30.34974186', 'longitude': '-97....",525N,MJ29,7,2019-07-30T11:08:30.000
79828,19-00288458,DOCKMOBI,Dockless Mobility,Transportation,Spot311 Interface,Closed,2019-07-30T11:04:37.000,2019-07-30T09:10:19.000,2019-07-30T11:04:37.000,"716 CONGRESS AVE, AUSTIN, TX 78701",...,TRAVIS,3114394.2485623,10071339.0020535,30.26967027,-97.74250614,"{'latitude': '30.26967027', 'longitude': '-97....",585S,MJ22,9,2019-07-30T11:04:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260696,18-00220040,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2018-09-26T14:42:44.000,2018-07-10T09:05:35.000,2018-09-26T14:42:44.000,"122 FREDERICK ST, AUSTIN, TX 78704",...,TRAVIS,3108991.74713961,10057136.0007393,30.23097313,-97.76065557,"{'latitude': '30.23097313', 'longitude': '-97....",614Z,MH19,3,2018-09-26T14:42:44.000
262317,18-00215309,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2018-09-26T14:46:37.000,2018-07-06T11:42:03.000,2018-09-26T14:46:37.000,"901 CHICON ST, AUSTIN, TX 78702",...,TRAVIS,3121195.25,10069591,30.26442801,-97.72109845,"{'latitude': '30.26442801', 'longitude': '-97....",585Y,MK22,1,2018-09-26T14:46:37.000
262603,18-00214399,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2018-09-26T14:50:43.000,2018-07-05T18:11:22.000,2018-09-26T14:50:43.000,"1609 ENFIELD RD, AUSTIN, TX 78703",...,TRAVIS,3109102.24901859,10075988.0002395,30.28278746,-97.75892441,"{'latitude': '30.28278746', 'longitude': '-97....",584M,MH23,9,2018-09-26T14:50:43.000
263303,18-00211272,DOCKMOBI,Dockless Mobility,Transportation,Phone,Closed,2019-01-09T10:09:41.000,2018-07-04T11:21:35.000,2019-01-09T10:09:41.000,"1172 WALLER ST, AUSTIN, TX 78702",...,TRAVIS,3118851.27746993,10071756.9384632,30.27053294,-97.72836041,"{'latitude': '30.27053294', 'longitude': '-97....",585U,MK22,1,2019-01-09T10:09:41.000


In [23]:
clean_df["SR Location Zip Code"]

361       78705
365       78701
470       78751
543       78704
547       78701
          ...  
260696    78704
262317    78702
262603    78703
263303    78702
266274    78701
Name: SR Location Zip Code, Length: 8799, dtype: object

In [29]:
# Convert dataframe to CSV 
output_data_file = "/resources/311data.csv"
clean_df.to_csv(output_data_file)

FileNotFoundError: [Errno 2] No such file or directory: '../resources/311data.csv'